# Chains

> Chains-based functions for PDFs.

In [ ]:
# | default_exp pdf.chains

In [ ]:
# | export

from langchain_ray.imports import *
from langchain_ray.chains import *
from langchain_ray.pdf.utils import *

In [ ]:
# | export


def pdf_docs_chain(
    chunk_size=200, chunk_overlap=20, verbose=False, input_key="pdf_folder", output_key="df"
):
    pdf_chain = transform_chain(create_pdf_df, input_key=input_key)
    docs_chain = transform_chain(
        df_pdf_docs,
        transform_kwargs={"chunk_size": chunk_size, "chunk_overlap": chunk_overlap},
    )
    return SimpleSequentialChain(
        chains=[pdf_chain, docs_chain],
        input_key=input_key,
        output_key=output_key,
        verbose=verbose,
    )


def pdf_faiss_chain(
    ems_model,
    index_folder,
    index_name,
    input_key="pdf_folder",
    output_key="df",
    chunk_size=200,
    chunk_overlap=20,
    docs_block_size=1500,
    cats_model=None,
    hf_ems=None,
    ems_folder=None,
    verbose=False,
):
    chain1 = pdf_docs_chain(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, input_key=input_key
    )
    index_chains = []
    if cats_model is not None:
        cats_chain = transform_chain(
            df_docs_cat,
            transform_kwargs={"cats_model": cats_model},
        )
        index_chains.append(cats_chain)
    if ems_folder is not None and hf_ems is not None:
        ems_chain = transform_chain(
            df_docs_ems,
            transform_kwargs={
                "ems_model": hf_ems,
                "ems_folder": ems_folder,
            },
        )
        index_chains.append(ems_chain)

    faiss_chain = transform_chain(
        df_to_faiss,
        transform_kwargs={
            "ems_model": ems_model,
            "index_folder": index_folder,
            "index_name": index_name,
        },
    )
    index_chains.append(faiss_chain)
    chain2 = ray_chain(
        SimpleSequentialChain(chains=index_chains),
        block_size=docs_block_size,
        cuda=True,
    )
    return SimpleSequentialChain(
        chains=[chain1, chain2], input_key=input_key, output_key=output_key, verbose=verbose
    )


def index_query_chain(
    ems_model,
    index_folder,
    index_name,
    input_key="query",
    output_key="search_results",
    k=2,
    block_size=10,
    verbose=False,
):
    q_df_chain = transform_chain(
        create_idx_q_df,
        input_key=input_key,
        transform_kwargs={"index_folder": index_folder, "index_name": index_name},
    )

    search_chain = transform_chain(
        lambda df: df.apply(df_search_faiss, axis=1, ems_model=ems_model, k=k),
    )

    res_chain = transform_chain(
        lambda df: sorted(flatten_list(df.results), key=lambda x: x[1]),
    )

    return ray_chain(
        SimpleSequentialChain(
            chains=[q_df_chain, search_chain, res_chain],
            input_key=input_key,
            output_key=output_key,
            verbose=verbose,
        ),
        block_size=block_size,
        cuda=True,
    )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()